In [2]:
import numpy as np

class mdp():
    def __init__(self, prob_, r_):
        self.prob = prob_
        self.r = r_
        
    def reward(self, n):
        if n==1:
            return self.prob*self.r[0] + (1-self.prob)*self.r[1]
        elif n==2:
            return self.prob*self.r[2] + (1-self.prob)*self.r[3]
        elif n<=5:
            return self.r[n+1]
        else:
            raise ValueError("invalid n")
        
    def delta_E(self, n, V, gamma=1):
        """
        delta_E[n] = (r1 + gamma*r2 + gamma^2*r3 + ... gamma^(n-1)*rn + V[n] - V0)
        """
        error = 0
        for i in range(1, min(n+1,6)):
            error += self.reward(i) * gamma**(n-1)
        if n==1:
            error += (self.prob*V[1] + (1-self.prob)*V[2])*gamma**n
        elif n<=5:
            error += V[n+1]*gamma**n
        # for we can imagine there is an absorbing states after n=5, so V[n]=0 for n>=6
        error -= V[0]
        return error
        
        
probToState = 0.81
rewards = [7.9, -5.1, 2.5, -7.2, 9.0, 0.0, 1.6]
valueEstimates = [0.0, 4.0, 25.7, 0.0, 20.1, 12.2, 0.0]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
delta_E

{1: 13.552999999999999,
 2: 6.087000000000002,
 3: 35.187000000000005,
 4: 27.287000000000003,
 5: 16.687000000000005,
 6: 16.687000000000005}

\begin{align}
\Delta V(\lambda) &= \sum_{k=1}^{\inf} (1-\lambda) \lambda^{k-1} \Delta E_k \\
& = (1-\lambda) \Delta E_1 + (1-\lambda)\lambda \Delta E_2 + (1-\lambda)\lambda^2 \Delta E_3 + (1-\lambda)\lambda^3 \Delta E_4 + (1-\lambda)\lambda^4 \Delta E_5 + \sum_{k=6}^{\inf} (1-\lambda) \lambda^{k-1} \Delta E_6 \\
& = (1-\lambda) \Delta E_1 + (1-\lambda)\lambda \Delta E_2 + (1-\lambda)\lambda^2 \Delta E_3 + (1-\lambda)\lambda^3 \Delta E_4 + (1-\lambda)\lambda^4 \Delta E_5 + \lambda^5 \Delta E_6
\end{align}

since for $k>=6$, $E_k = E_6$, the infinit sum reduce to $\lambda^5 \Delta E_6$

\begin{equation}
\Delta V(1) = \Delta E_6
\end{equation}

solve for $\Delta V(\lambda) = \Delta V(1)$, which becomes a polynomial equation of $\lambda$

In [3]:
def e(lm, delta_E):
    return (1-lm)*delta_E[1] + (1-lm)*lm*delta_E[2] + (1-lm)*lm**2*delta_E[3] + (1-lm)*lm**3*delta_E[4] + (1-lm)*lm**4*delta_E[5] +lm**5*delta_E[6] - delta_E[6]

In [4]:
from scipy.optimize import fsolve
fsolve(e, x0=0.5, args=delta_E)

array([0.6227695])

In [5]:
# test case 2
probToState = 0.22
valueEstimates = [0.0, -5.2, 0.0, 25.4, 10.6, 9.2, 12.3]
rewards = [-2.4, 0.8, 4.0, 2.5, 8.6, -6.4, 6.1]
my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
print(delta_E)
fsolve(e, x0=0.5, args=delta_E)

{1: -1.048, 2: 28.326, 3: 22.125999999999998, 4: 14.325999999999999, 5: 23.526, 6: 11.225999999999999}


array([0.49567142])

In [6]:
# test case 3
probToState = 0.64
valueEstimates = [0.0, 4.9, 7.8, -2.3, 25.5, -10.2, -6.5]
rewards = [-2.4, 9.6, -7.8, 0.1, 3.4, -2.1, 7.9]
my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
print(delta_E)
fsolve(e, x0=0.5, args=delta_E)

{1: 7.864, 2: -5.336, 3: 25.864, 4: -11.936, 5: -0.3360000000000003, 6: 6.164}


array([0.20550276])

### quiz

In [7]:
probToState=0.15
valueEstimates=[0.0,0,4.1,17.4,17.4,21.8,5.7]
rewards=[4.2,-1.2,1.3,5.9,7.4,-2.1,0.1]
my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.33621109])

In [8]:
probToState=0.31

valueEstimates=[0.0,22.1,20.4,8.6,-4.1,0,0.0]

rewards=[7.0,3.7,5.2,-0.3,-4.4,7.7,9.3]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.45006241])

In [9]:
probToState=0.25

valueEstimates=[0.0,14.4,5.5,0,17.2,3.1,22.2]

rewards=[-4.7,6.5,-3,8.6,0,6.5,-0.4]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.60348186])

In [10]:
probToState=0.79

valueEstimates=[0.0,0,3.8,25,0,20.5,16.9]

rewards=[6.5,3.1,-0.6,1.6,0,9.3,-1.0]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.40813957])

In [11]:
probToState=0.46

valueEstimates=[0.0,13,0,24.3,19.3,-3.7,11.7]

rewards=[0.9,-2.2,4.2,-1.4,1,0,7.2]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.18796712])

In [12]:
probToState=0.55

valueEstimates=[0.0,0.2,13,19,22.1,17.5,0.0]

rewards=[9.8,0,6.6,5.8,5.7,-0.5,1.5]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.33153026])

In [13]:
probToState=1.0

valueEstimates=[0.0,0,-2.3,17.1,20.7,0,16.8]

rewards=[3.1,-3.9,3.2,9.8,1.1,1.1,-0.5]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.23936997])

In [14]:
probToState=0.72
valueEstimates=[0.0,14.1,0,-2,15.9,0,3.3]
rewards=[-3.5,0.9,-4.8,8.2,-2,6.1,5.4]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.16619674])

In [15]:
probToState=0.73
valueEstimates=[0.0,12.9,16.7,0,0,-2.3,0.4]
rewards=[8.3,-2,-0.1,-2.9,5.1,6.8,2.7]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.01236561])

In [16]:
probToState=0.09
valueEstimates=[0.0,0,14,4.7,7.9,13.8,19.0]
rewards=[-0.4,-1.1,9.7,7.4,1.6,0.5,3.6]

my_mdp = mdp(probToState, rewards)
delta_E = {n: my_mdp.delta_E(n, valueEstimates) for n in range(1,7)}
fsolve(e, x0=0.5, args=delta_E)

array([0.31486309])